In [1]:
import numpy as np
import random
import tensorflow as tf
from sklearn import preprocessing
import pandas as pd
from sklearn.preprocessing import Imputer, LabelEncoder, StandardScaler, scale
df = pd.read_csv('./baseline_data.csv',header=0)

/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2162) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.shape

(628, 2163)

In [4]:
Catdata = df.loc[0:,'PTETHCAT':'APOE4'].join(df['PTGENDER']).join(df['imputed_genotype'])
Numdata = df.loc[0:,:'Thickness..thickinthehead..2035'].join(df['AGE']).join(df['PTEDUCAT']).join(df['MMSE'])
labels = df['DX.bl']

In [5]:
Catdata.shape

(628, 5)

In [6]:
Numdata.shape

(628, 2153)

In [26]:
for i in Catdata.columns:
    the_value = str(Catdata[i].mode().values[0])
    Catdata[i].replace('NaN',the_value,inplace = True)
    dummy_data = pd.get_dummies(Catdata[i], prefix=i+"_", drop_first=True)
    Catdata = pd.concat([Catdata, dummy_data], axis=1)
    Catdata.drop(i, axis=1, inplace=True)
#for i in Numdata.columns:
    #the_value = str(Numdata[i].median().values[0])
    #Numdata[i].replace('NaN',the_value,inplace = True)

In [27]:
le = LabelEncoder()
le.fit(labels)
labels = le.transform(labels)
labels = labels.astype('float')

In [28]:
from sklearn.utils import resample
data = Numdata.join(Catdata)
data['labels'] = labels
#data = resample(data,n_samples=628, random_state = 2018)
data.head()

,area.1002,area.1003,area.1005,area.1006,area.1007,area.1008,area.1009,area.1011,area.1012,area.1013,...,MMSE,PTETHCAT__Not Hisp/Latino,PTETHCAT__Unknown,PTRACCAT__Black,PTRACCAT__White,APOE4__1,APOE4__2,PTGENDER__Male,imputed_genotype__True,labels
0,612.577638,2514.366512,1652.050796,731.718725,3794.131041,5012.960558,4951.994756,5666.361234,2568.454160,3147.024829,...,28,1,0,0,1,1,0,1,1,1.0
1,735.292087,2435.629408,1947.966106,721.230657,4101.035394,4469.814924,4002.936490,5427.875646,2778.342103,3907.628206,...,29,1,0,0,1,0,0,0,1,1.0
2,1080.976588,2190.801306,1613.620315,636.078912,5146.969073,6192.609394,4640.889149,6854.023127,3282.902615,3677.408865,...,22,1,0,0,1,0,1,1,0,0.0
3,840.850798,2293.601605,1599.807666,729.344575,3351.924971,4231.417941,3991.795466,5047.106646,2599.222056,3339.590461,...,30,1,0,0,1,0,0,0,1,1.0
4,592.882184,1827.195664,1479.821407,535.558408,3459.934118,5063.103074,3583.954659,4120.492969,2462.352998,2737.467877,...,27,1,0,0,1,1,0,0,1,2.0


In [29]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = ([],[],[],[])
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,:'imputed_genotype__True'], data['labels'], test_size = 0.2, random_state = 2018)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = scaler.transform(X_test)

In [30]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160
0,-0.674453,-0.679446,0.081322,-0.746656,-0.977341,-1.709378,-1.631854,-1.401668,-0.883885,-1.484730,...,0.130844,1.170398,0.135113,-0.077537,-0.224074,0.277945,-0.777067,-0.336277,-1.150684,0.597291
1,-1.858632,-0.706904,-0.762047,0.004010,-1.619124,-0.968373,-1.640428,-0.416259,-0.821928,-1.165071,...,-1.536249,0.789620,0.135113,-0.077537,4.462809,-3.597839,1.286890,-0.336277,0.869048,-1.674225
2,0.377906,1.295648,-0.489598,-0.419993,0.603891,-0.077964,0.355101,-0.908562,0.075644,-0.327764,...,0.130844,0.408843,0.135113,-0.077537,-0.224074,0.277945,-0.777067,-0.336277,-1.150684,-1.674225
3,-0.043536,-1.294023,-1.172906,-0.558682,-0.338841,-0.591348,-0.510148,-1.332073,-0.478782,-0.831695,...,-1.202831,-1.114267,0.135113,-0.077537,-0.224074,0.277945,-0.777067,-0.336277,-1.150684,0.597291
4,-0.069628,-0.025465,-0.344510,-1.905992,1.078735,0.456331,0.645210,0.291629,2.313630,1.157744,...,-2.536505,-1.114267,0.135113,-0.077537,-0.224074,0.277945,-0.777067,-0.336277,0.869048,0.597291


In [62]:
trainX = np.array(X_train)
trainyy = np.array(y_train).reshape(-1,1)
trainy = np.zeros([trainyy.shape[0],3])
for i in range(trainyy.shape[0]):
    trainy[i, int(trainyy[i])] = 1
testX = np.array(X_test)
testyy = np.array(y_test).reshape(-1,1)
testy = np.zeros([testyy.shape[0],3])
for i in range(testyy.shape[0]):
    testy[i, int(testyy[i])] = 1

In [79]:
x = tf.placeholder(tf.float32,[None,2161])
y = tf.placeholder(tf.float32,[None,3])

W1 = tf.Variable(tf.truncated_normal([2161,200],stddev=0.1))
b1 = tf.Variable(tf.zeros([200])+0.1)
z1 = tf.matmul(x,W1)+b1
h1 = tf.nn.relu(z1)

W2 = tf.Variable(tf.truncated_normal([200,50],stddev=0.1))
b2 = tf.Variable(tf.zeros([50])+0.1)
z2 = tf.matmul(z1,W2)+b2
h2 = tf.nn.tanh(z2)

W3 = tf.Variable(tf.truncated_normal([50,10],stddev=0.1))
b3 = tf.Variable(tf.zeros([10])+0.1)
z3 = tf.matmul(h2,W3)+b3
h3 = tf.nn.tanh(z3)

W4 = tf.Variable(tf.truncated_normal([10,3],stddev=0.1))
b4 = tf.Variable(tf.zeros([3])+0.1)
z4 = tf.matmul(h3,W4)+b4

y_prediction = tf.nn.softmax(z4)

#loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_prediction), reduction_indices=[1]))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = y_prediction))
#loss = tf.reduce_mean((y-y_prediction)**2)

#Train = tf.train.GradientDescentOptimizer(0.0005).minimize(loss) #DOM:0.00001
Train = tf.train.MomentumOptimizer(learning_rate=0.5,momentum = 0.9).minimize(loss)
#Train = tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

correct_p = tf.equal(tf.argmax(y,1),tf.argmax(y_prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_p,tf.float32))
prediction=tf.argmax(y_prediction,1)

# using validation set to determine hyperparameters.
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(1500):
        sess.run(Train,feed_dict={x:trainX,y:trainy})
        acc = sess.run(accuracy,feed_dict={x:trainX, y:trainy})
        tacc = sess.run(accuracy,feed_dict={x:testX, y:testy})
        cost = sess.run(loss,feed_dict={x:trainX,y:trainy})
        #print('batch: '+ str(batch)+',node: '+str(node)+', learning rate: '+str(lr))
        print('Iter'+str(epoch+1)+',Training loss: '+str(cost)+', Accuracy: '+str(acc)+',TAccuracy:'+str(tacc))#+',f1_score:'+str(f_score))
    best = sess.run([prediction],feed_dict={x:testX})
    print(best)

Iter1,Training loss: 1.0956194, Accuracy: 0.36055776,TAccuracy:0.3888889
Iter2,Training loss: 1.0858626, Accuracy: 0.42629483,TAccuracy:0.44444445
Iter3,Training loss: 1.0733498, Accuracy: 0.49203187,TAccuracy:0.5
Iter4,Training loss: 1.0592997, Accuracy: 0.53386456,TAccuracy:0.52380955
Iter5,Training loss: 1.0439098, Accuracy: 0.5278884,TAccuracy:0.53174603
Iter6,Training loss: 1.0268135, Accuracy: 0.52988046,TAccuracy:0.515873
Iter7,Training loss: 1.0071383, Accuracy: 0.5458167,TAccuracy:0.52380955
Iter8,Training loss: 0.983598, Accuracy: 0.5976096,TAccuracy:0.53968257
Iter9,Training loss: 0.9550369, Accuracy: 0.64143425,TAccuracy:0.56349206
Iter10,Training loss: 0.9205682, Accuracy: 0.69123507,TAccuracy:0.5793651
Iter11,Training loss: 0.88009465, Accuracy: 0.7410359,TAccuracy:0.5793651
Iter12,Training loss: 0.8345541, Accuracy: 0.7868526,TAccuracy:0.5555556
Iter13,Training loss: 0.78629917, Accuracy: 0.8286853,TAccuracy:0.515873
Iter14,Training loss: 0.7406948, Accuracy: 0.85856575,

Iter112,Training loss: 0.55849785, Accuracy: 0.9920319,TAccuracy:0.46825397
Iter113,Training loss: 0.5580338, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter114,Training loss: 0.55763614, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter115,Training loss: 0.5574178, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter116,Training loss: 0.5573156, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter117,Training loss: 0.55727065, Accuracy: 0.9940239,TAccuracy:0.47619048
Iter118,Training loss: 0.5572483, Accuracy: 0.9940239,TAccuracy:0.47619048
Iter119,Training loss: 0.55723447, Accuracy: 0.9940239,TAccuracy:0.47619048
Iter120,Training loss: 0.557223, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter121,Training loss: 0.5572134, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter122,Training loss: 0.55720466, Accuracy: 0.9940239,TAccuracy:0.46825397
Iter123,Training loss: 0.55719715, Accuracy: 0.9940239,TAccuracy:0.47619048
Iter124,Training loss: 0.55719024, Accuracy: 0.9940239,TAccuracy:0.47619048
Iter125,Training lo

Iter221,Training loss: 0.5551164, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter222,Training loss: 0.5551152, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter223,Training loss: 0.5551141, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter224,Training loss: 0.55511343, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter225,Training loss: 0.555113, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter226,Training loss: 0.5551128, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter227,Training loss: 0.5551123, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter228,Training loss: 0.55511165, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter229,Training loss: 0.55511093, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter230,Training loss: 0.55511034, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter231,Training loss: 0.5551099, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter232,Training loss: 0.5551096, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter233,Training loss: 0.5551093, Accuracy: 0.99601597,TAccuracy:0.46825397
Iter234,T

Iter332,Training loss: 0.5550901, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter333,Training loss: 0.5550901, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter334,Training loss: 0.55509, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter335,Training loss: 0.5550899, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter336,Training loss: 0.55508983, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter337,Training loss: 0.5550898, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter338,Training loss: 0.55508965, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter339,Training loss: 0.55508953, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter340,Training loss: 0.5550895, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter341,Training loss: 0.5550894, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter342,Training loss: 0.5550893, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter343,Training loss: 0.55508924, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter344,Training loss: 0.5550891, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter345,Tr

Iter446,Training loss: 0.5550823, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter447,Training loss: 0.55508226, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter448,Training loss: 0.5550822, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter449,Training loss: 0.5550821, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter450,Training loss: 0.555082, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter451,Training loss: 0.555082, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter452,Training loss: 0.55508196, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter453,Training loss: 0.5550819, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter454,Training loss: 0.55508184, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter455,Training loss: 0.5550818, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter456,Training loss: 0.5550817, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter457,Training loss: 0.5550817, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter458,Training loss: 0.5550817, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter459,Tra

...

Iter1005,Training loss: 0.5550706, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1006,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1007,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1008,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1009,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1010,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1011,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1012,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1013,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1014,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1015,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1016,Training loss: 0.55507046, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1017,Training loss: 0.5550704, Accuracy: 0.99601597,TAccuracy

Iter1113,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1114,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1115,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1116,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1117,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1118,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1119,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1120,Training loss: 0.55506945, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1121,Training loss: 0.5550694, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1122,Training loss: 0.5550694, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1123,Training loss: 0.5550694, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1124,Training loss: 0.5550694, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1125,Training loss: 0.5550693, Accuracy: 0.99601597,TAccuracy:0.

Iter1222,Training loss: 0.55506843, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1223,Training loss: 0.55506843, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1224,Training loss: 0.55506843, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1225,Training loss: 0.55506843, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1226,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1227,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1228,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1229,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1230,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1231,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1232,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1233,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.47619048
Iter1234,Training loss: 0.5550684, Accuracy: 0.99601597,TAccuracy:0.4761

Iter1333,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1334,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1335,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1336,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1337,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1338,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1339,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1340,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1341,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1342,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1343,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1344,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1345,Training loss: 0.5550678, Accuracy: 0.99601597,TAccuracy:0.48412699

Iter1443,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1444,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1445,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1446,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1447,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1448,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1449,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1450,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1451,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1452,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1453,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1454,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699
Iter1455,Training loss: 0.5550671, Accuracy: 0.99601597,TAccuracy:0.48412699

In [80]:
from sklearn.metrics import f1_score
best = best[0]
besty = np.zeros([best.shape[0],3])
for i in range(best.shape[0]):
    besty[i, int(best[i])] = 1
f1_score(testy,besty,average='macro')

0.45593337959750163